In [46]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

In [2]:
pred_cols = ["Mean_BMI","Median_BMI","Unmet_Need_Rate","Under5_Mortality_Rate","Skilled_Birth_Attendant_Rate","Stunted_Rate"]

In [3]:
def mcrmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return np.average(rmse)

In [5]:
df = pd.read_parquet('../../data/top_features_data.parquet.gzip')

In [10]:
df

,Es_min_max@CAS/IGSNRR/PML/V2&timestamped,basic_demographic_characteristics_min_max@CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics&timestamped,national_identifier_grid@CIESIN/GPWv411/GPW_National_Identifier_Grid,national_identifier_grid_max@CIESIN/GPWv411/GPW_National_Identifier_Grid,national_identifier_grid_min@CIESIN/GPWv411/GPW_National_Identifier_Grid,urban-coverfraction_mean@COPERNICUS/Landcover/100m/Proba-V-C3/Global&timestamped,SO2_column_number_density_15km_max_max@COPERNICUS/S5P/NRTI/L3_SO2&timestamped,solar_azimuth_angle_max_max@COPERNICUS/S5P/NRTI/L3_SO2&timestamped,ozone_tropospheric_mixing_ratio_median@COPERNICUS/S5P/OFFL/L3_O3_TCL&timestamped,ozone_tropospheric_vertical_column_median@COPERNICUS/S5P/OFFL/L3_O3_TCL&timestamped,...,onehotencoder__URBAN_RURA_U,remainder__DHSYEAR,remainder__LATNUM,remainder__LONGNUM,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
DHSID,,,,,,,,,,,,,,,,,,,,,
AL200800000001,0.079005,0.533688,8.0,8,8,15.942918,0.001929,179.557310,21.800858,-0.994597,...,0.0,2008.0,40.822650,19.838322,24.12,25.28,50.00,9.68,100.00,20.00
AL200800000002,0.090663,0.165650,8.0,8,8,1.059678,0.001774,179.592150,21.800858,-0.994597,...,0.0,2008.0,40.696846,20.007555,23.04,21.98,7.69,8.33,66.67,0.00
AL200800000003,0.103369,0.444329,8.0,8,8,3.955272,0.001707,179.530600,21.800858,-0.994597,...,0.0,2008.0,40.750040,19.974262,NaN,NaN,NaN,NaN,NaN,NaN
AL200800000004,0.085710,0.814697,8.0,8,8,10.031118,0.001929,179.768220,21.800858,-0.994597,...,0.0,2008.0,40.798930,19.863338,26.74,26.57,7.69,2.86,100.00,0.00
AL200800000005,0.096352,0.541894,8.0,8,8,0.901964,0.001859,179.618840,21.800858,-0.994597,...,0.0,2008.0,40.746124,19.843885,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZW201500000396,0.002424,0.000000,716.0,716,716,80.522360,0.000402,-17.357134,53.740124,0.011052,...,1.0,2015.0,-17.915289,31.156115,23.16,22.14,33.33,2.04,78.95,25.00
ZW201500000397,0.038481,0.000000,716.0,716,716,0.298818,0.000336,-16.403381,53.757790,0.010933,...,0.0,2015.0,-18.379501,31.872288,24.33,22.61,11.11,1.75,76.19,22.22
ZW201500000398,0.002700,0.000000,716.0,716,716,0.000000,0.000681,-16.150686,53.238773,0.011527,...,0.0,2015.0,-16.660612,29.850649,23.70,21.44,10.53,4.88,68.00,17.65


In [37]:
y = df[['Mean_BMI', 'Median_BMI']]
X = df.drop(columns=pred_cols)

# get rows in y that have all null values
null_rows = y[y.isna().any(axis=1)].index
X.drop(index=null_rows, inplace=True)
y.drop(index=null_rows, inplace=True)

/var/folders/qb/stptmtq13_9gt1v1pzd4g4s00000gn/T/ipykernel_6943/2044479735.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.drop(index=null_rows, inplace=True)


In [41]:
# KFold training of Random Forest model

kf = KFold(n_splits=2, shuffle=True, random_state=42)
scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    rf = RandomForestRegressor(n_estimators=6000, max_depth=20, max_features=0.5, random_state=42, n_jobs=-1, verbose=1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    scores.append(mcrmse(y_test, y_pred))

score = np.average(scores)
score

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed: 15.0min
[Parallel(n_jobs=-1)]: Done 6000 out of 6000 | elapsed: 17.7min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8

2.1090889465957785

In [42]:
sub = pd.read_csv('../../data/sample submission.csv', index_col='DHSID')
df.drop(columns=pred_cols, inplace=True)
test = sub.join(df, how='left')
test.drop(columns=pred_cols, inplace=True)
test

,Es_min_max@CAS/IGSNRR/PML/V2&timestamped,basic_demographic_characteristics_min_max@CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics&timestamped,national_identifier_grid@CIESIN/GPWv411/GPW_National_Identifier_Grid,national_identifier_grid_max@CIESIN/GPWv411/GPW_National_Identifier_Grid,national_identifier_grid_min@CIESIN/GPWv411/GPW_National_Identifier_Grid,urban-coverfraction_mean@COPERNICUS/Landcover/100m/Proba-V-C3/Global&timestamped,SO2_column_number_density_15km_max_max@COPERNICUS/S5P/NRTI/L3_SO2&timestamped,solar_azimuth_angle_max_max@COPERNICUS/S5P/NRTI/L3_SO2&timestamped,ozone_tropospheric_mixing_ratio_median@COPERNICUS/S5P/OFFL/L3_O3_TCL&timestamped,ozone_tropospheric_vertical_column_median@COPERNICUS/S5P/OFFL/L3_O3_TCL&timestamped,...,onehotencoder__DHSCC_DR,onehotencoder__DHSCC_ET,onehotencoder__DHSCC_HT,onehotencoder__DHSCC_MZ,onehotencoder__DHSCC_TZ,onehotencoder__URBAN_RURA_R,onehotencoder__URBAN_RURA_U,remainder__DHSYEAR,remainder__LATNUM,remainder__LONGNUM
DHSID,,,,,,,,,,,,,,,,,,,,,
AL200800000003,0.103369,0.444329,8.00000,8.0,8.0,3.955272,0.001707,179.530600,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2008.0,40.750040,19.974262
AL200800000005,0.096352,0.541894,8.00000,8.0,8.0,0.901964,0.001859,179.618840,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2008.0,40.746124,19.843885
AL200800000007,0.098821,3.778325,8.00000,8.0,8.0,12.266235,0.001827,179.544780,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,40.701607,19.989952
AL200800000008,0.105989,8.942595,8.00000,8.0,8.0,50.352406,0.001986,179.544780,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,40.695984,19.965063
AL200800000009,0.101052,8.941481,8.00000,8.0,8.0,34.656734,0.001986,179.557740,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,40.698685,19.981623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZW201500000382,0.012040,0.000000,716.00000,716.0,716.0,0.000000,0.000443,-15.686703,48.700848,0.010540,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2015.0,-19.939451,31.822948
ZW201500000383,0.000000,0.000000,716.00000,716.0,716.0,-1.000000,0.000312,-16.927180,49.692770,0.010532,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2015.0,-19.851952,28.240479
ZW201500000386,0.000000,0.000000,-19.36183,716.0,710.0,0.000000,0.000339,-16.038418,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2015.0,-22.126260,29.639608


In [43]:
model = RandomForestRegressor(n_estimators=6000, max_depth=20, max_features=0.5, random_state=42, n_jobs=-1, verbose=1)
model.fit(X, y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 23.1min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed: 28.4min
[Parallel(n_jobs=-1)]: Done 6000 out of 6000 | elapsed: 34.0min finished


ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [47]:
# impute test values
imputer = KNNImputer(n_neighbors=75)
test = pd.DataFrame(imputer.fit_transform(test), columns=test.columns, index=test.index)
test

,Es_min_max@CAS/IGSNRR/PML/V2&timestamped,basic_demographic_characteristics_min_max@CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics&timestamped,national_identifier_grid@CIESIN/GPWv411/GPW_National_Identifier_Grid,national_identifier_grid_max@CIESIN/GPWv411/GPW_National_Identifier_Grid,national_identifier_grid_min@CIESIN/GPWv411/GPW_National_Identifier_Grid,urban-coverfraction_mean@COPERNICUS/Landcover/100m/Proba-V-C3/Global&timestamped,SO2_column_number_density_15km_max_max@COPERNICUS/S5P/NRTI/L3_SO2&timestamped,solar_azimuth_angle_max_max@COPERNICUS/S5P/NRTI/L3_SO2&timestamped,ozone_tropospheric_mixing_ratio_median@COPERNICUS/S5P/OFFL/L3_O3_TCL&timestamped,ozone_tropospheric_vertical_column_median@COPERNICUS/S5P/OFFL/L3_O3_TCL&timestamped,...,onehotencoder__DHSCC_DR,onehotencoder__DHSCC_ET,onehotencoder__DHSCC_HT,onehotencoder__DHSCC_MZ,onehotencoder__DHSCC_TZ,onehotencoder__URBAN_RURA_R,onehotencoder__URBAN_RURA_U,remainder__DHSYEAR,remainder__LATNUM,remainder__LONGNUM
DHSID,,,,,,,,,,,,,,,,,,,,,
AL200800000003,0.103369,0.444329,8.00000,8.0,8.0,3.955272,0.001707,179.530600,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2008.0,40.750040,19.974262
AL200800000005,0.096352,0.541894,8.00000,8.0,8.0,0.901964,0.001859,179.618840,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2008.0,40.746124,19.843885
AL200800000007,0.098821,3.778325,8.00000,8.0,8.0,12.266235,0.001827,179.544780,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,40.701607,19.989952
AL200800000008,0.105989,8.942595,8.00000,8.0,8.0,50.352406,0.001986,179.544780,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,40.695984,19.965063
AL200800000009,0.101052,8.941481,8.00000,8.0,8.0,34.656734,0.001986,179.557740,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2008.0,40.698685,19.981623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZW201500000382,0.012040,0.000000,716.00000,716.0,716.0,0.000000,0.000443,-15.686703,48.700848,0.010540,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2015.0,-19.939451,31.822948
ZW201500000383,0.000000,0.000000,716.00000,716.0,716.0,-1.000000,0.000312,-16.927180,49.692770,0.010532,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2015.0,-19.851952,28.240479
ZW201500000386,0.000000,0.000000,-19.36183,716.0,710.0,0.000000,0.000339,-16.038418,21.800858,-0.994597,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2015.0,-22.126260,29.639608


In [48]:
preds = model.predict(test)
preds

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    2.2s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    3.8s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    5.8s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    8.1s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:   10.9s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:   14.1s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:   17.8s
[Parallel(n_jobs=8)]: Done 6000 out of 6000 | elapsed:   22.1s finished


array([[25.04631225, 24.21387929],
       [25.20963403, 24.44330392],
       [25.15106475, 24.73511195],
       ...,
       [26.2663281 , 25.64278926],
       [25.76807587, 24.89131332],
       [25.3250718 , 24.52480722]])

In [49]:
final = pd.DataFrame(preds, columns=['Mean_BMI', 'Median_BMI'], index=sub.index)
final

,Mean_BMI,Median_BMI
DHSID,,
AL200800000003,25.046312,24.213879
AL200800000005,25.209634,24.443304
AL200800000007,25.151065,24.735112
AL200800000008,25.044528,24.595279
AL200800000009,24.947219,24.536175
...,...,...
ZW201500000382,23.596052,22.802582
ZW201500000383,23.348384,22.632835
ZW201500000386,26.266328,25.642789


In [50]:
final.to_parquet('./bmi.parquet.gzip', compression='gzip')